In [2]:
import pandas as pd
import numpy as np

In [3]:
file_dir = 'data/'

In [4]:
# import excel file and add year column and values.
# each excel sheet is imported and put into a dictionary with the key being the respective year
sheets = ['2021','2019','2018','2017','2016','2015','2014']
dat = {}
for s in sheets:
    dat[s] = pd.read_csv(f"{file_dir}{s} data.csv", index_col=0)
    dat[s]['Year'] = int(s)

In [7]:
# compress all the data into the same dataframe and delete original dictionary of dataframes
df = dat[sheets[0]]
for s in sheets[1:]:
    df = df.append(dat[s])
df = df.reset_index()
df = df.drop(['index'],axis=1)

In [8]:
#set default optimal rank to be 0
df['optimalrank'] = 0

In [24]:
def optimize_score(df):
    for i in range(0,len(df)):
        #print(i)
        
# 0 wins stays 0
        if (df.at[i, 'Number of Tournament Wins'] == 0):
            df.at[i,'optimalrank'] = 0
            
#6 wins
        if (df.at[i, 'Number of Tournament Wins'] == 6):
            df.at[i,'optimalrank'] = 10
#5 wins
        if (df.at[i, 'Number of Tournament Wins'] == 5):
            df.at[i,'optimalrank'] = 9
#4 wins
        if (df.at[i, 'Number of Tournament Wins'] == 4):
            df.at[i,'optimalrank'] = 7
#3 wins
#ranking non-cinderella teams higher for 3 wins, as non-cinderella teams are much more likely to do better
        if ((df.at[i, 'Number of Tournament Wins'] == 3) and (df.at[i, 'Cinderella'] == 0)):
            df.at[i,'optimalrank'] = 6
            
        if ((df.at[i, 'Number of Tournament Wins'] == 3) and (df.at[i, 'Cinderella'] == 1)):
            df.at[i,'optimalrank'] = 5
# 2 wins 
#cinderella with 2 wins
        if ((df.at[i, 'Number of Tournament Wins'] == 2) and (df.at[i, 'Cinderella'] == 1)):
            df.at[i,'optimalrank'] = 3
# 1 win
#cinderella with 1 win
        if ((df.at[i, 'Number of Tournament Wins'] == 2) and (df.at[i, 'Cinderella'] == 1)):
            df.at[i,'optimalrank'] = 2
# 2 wins non-cinderella
        if ((df.at[i, 'Number of Tournament Wins'] == 2) and (df.at[i, 'Cinderella'] == 0)):
            df.at[i,'optimalrank'] = 1
    return df

In [25]:
#form list and continue to add values above 0 until we reach 10, then get rid of smallest value until at the
#end of going through all rows
def get_optimal_score(df):
    rank_list = []
    for i in range(0, len(df)):
        val = df.at[i, 'optimalrank'] #value at current rank level
        print("index: " + str(i) + " rank: " + str(val))
        #print("current list", list)
        listLength = len(rank_list)
        
        if (val > 0):
            
            #print("current list length: ", listLength)
            
            if (listLength < 10):
                rank_list.append(val)
                
            if (listLength >= 10):
                #find the smallest rank in the rank list, and then compare to see if the new value is larger than smallest
                smallest = rank_list[0]
                #print("smallest begins as: ", smallest)
                for element in rank_list:
                    #print("current element: " + str(element))
                    if (element < smallest):
                        #print("new smallest: ", smallest)
                        smallest = element
                
                #now to compare val to smallest
                if (val > smallest):
                    rank_list.remove(smallest)
                    rank_list.append(val)
                    print("should add " + str(val) + " to list")
                    print(rank_list)
    return rank_list


In [26]:
df.sort_values('Number of Tournament Wins', inplace=True, ascending=False)

df_o = optimize_score(df)
#get_optimal_score(df16)

In [28]:
df_o

,Team,Conference,Region,Cinderella,Conference Tournament Champion,Number of Tournament Wins,Made Tournament Previous Year,Game Count,Wins,Losses,...,Total Points,Average PPG,Total Opp Points,Average Opp PPG,Total Scoring Differential,Scoring Differential Per Game,Year,Wins Against Top 25 RPI Teams,Losses Against Top 25 RPI Teams,optimalrank
238,North Carolina,Atlantic Coast,South,0.0,0.0,6.0,1.0,34.0,27.0,7.0,...,2887.0,84.900,2401.0,70.600000,486.0,14.300000,2017,5.0,4.0,10
355,Duke,Atlantic Coast,South,0.0,0.0,6.0,1.0,33.0,29.0,4.0,...,2660.0,80.600,2164.0,65.600000,496.0,15.000000,2015,7.0,2.0,10
0,Baylor,B12,South,0.0,0.0,6.0,1.0,23.0,22.0,1.0,...,2025.0,84.375,1594.0,66.416664,431.0,17.958334,2021,NaN,NaN,10
467,UConn,AAC,East,0.0,0.0,6.0,0.0,34.0,26.0,8.0,...,2443.0,71.900,2146.0,63.100000,297.0,8.800000,2014,3.0,4.0,10
68,Virginia,ACC,South,0.0,0.0,6.0,1.0,32.0,29.0,3.0,...,2299.0,71.800,1762.0,55.100000,537.0,16.800000,2019,13.0,3.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,New Mexico St.,Western Athletic,Midwest,1.0,1.0,0.0,1.0,33.0,23.0,10.0,...,2260.0,68.500,1958.0,59.300000,302.0,9.200000,2015,0.0,2.0,0
377,North Dakota St.,Summit League,South,1.0,1.0,0.0,1.0,32.0,23.0,9.0,...,2065.0,64.500,1969.0,61.500000,96.0,3.000000,2015,0.0,0.0,0
378,North Florida,ASUN,South,1.0,1.0,0.0,0.0,34.0,23.0,11.0,...,2574.0,75.700,2304.0,67.800000,270.0,7.900000,2015,0.0,1.0,0
379,Northeastern,Colonial,Midwest,1.0,1.0,0.0,0.0,34.0,23.0,11.0,...,2332.0,68.600,2210.0,65.000000,122.0,3.600000,2015,0.0,0.0,0


In [29]:
df_o.to_csv("./processed_data/optimized_data.csv")

In [26]:
def sort_df(testDF):
    testDF_sorted = testDF.sort_values(["optimalrank"], ascending = False)
    print(testDF_sorted)
    return testDF_sorted

In [28]:
#currently not sorting within the function for some reason
def calc_optimal_score(rankedDF):
    total_score = 0
    rankedDF_sorted = rankedDF.sort_values(["optimalrank"], ascending = False)
    rankedDF_sorted = rankedDF_sorted.reset_index()
    rankedDF_sorted = rankedDF_sorted.drop(["index"], axis = 1)
    
    for i in range(10):
        print(i)
        total_score += (rankedDF_sorted.at[i, "Number of Tournament Wins"] * (10 - i))
        #print("unsorted wins: ", rankedDF.at[i, "Number of Tournament Wins"])
        print("Num tournament wins is: ", rankedDF_sorted.at[i, "Number of Tournament Wins"])
        print ("10 - i is ", (10-i))
        print("Total score: ", total_score)
        if (rankedDF_sorted.at[i, "Cinderella"] == 1):
            total_score += (5 * rankedDF_sorted.at[i, "Number of Tournament Wins"])
            print("Was cinderella so now total score is: ", total_score)
    print(total_score)
    return total_score
        

In [29]:
calc_optimal_score(df2014)

0
Num tournament wins is:  6.0
10 - i is  10
Total score:  60.0
1
Num tournament wins is:  5.0
10 - i is  9
Total score:  105.0
2
Num tournament wins is:  4.0
10 - i is  8
Total score:  137.0
3
Num tournament wins is:  4.0
10 - i is  7
Total score:  165.0
4
Num tournament wins is:  3.0
10 - i is  6
Total score:  183.0
5
Num tournament wins is:  3.0
10 - i is  5
Total score:  198.0
6
Num tournament wins is:  3.0
10 - i is  4
Total score:  210.0
7
Num tournament wins is:  3.0
10 - i is  3
Total score:  219.0
Was cinderella so now total score is:  234.0
8
Num tournament wins is:  2.0
10 - i is  2
Total score:  238.0
Was cinderella so now total score is:  248.0
9
Num tournament wins is:  2.0
10 - i is  1
Total score:  250.0
Was cinderella so now total score is:  260.0
260.0


260.0